In [1]:
import warnings
warnings.filterwarnings('ignore')
import librosa
import pandas as pd
import numpy as np
import os
from scipy.fftpack import fft

In [2]:
def getFiles(directory,typ='.mp3'):
    for file in os.listdir(directory):
        if file.endswith(typ):
            yield('{}/{}'.format(directory,file))
        else:
            continue

In [3]:
def getAudio(file):
    data, sampling_rate = librosa.load(file,res_type='kaiser_fast')
    return data, sampling_rate

In [4]:
df = pd.DataFrame(columns=['lang','data','sample_rate'])
for lang in os.listdir('../data/samples'):
    for file in getFiles('../data/samples/{}'.format(lang)):
        data, sample_rate = getAudio(file)
        df = df.append({'data':data,'lang':lang,'sample_rate':sample_rate}, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
df.head()

In [5]:
def getMFCC(row):
    mfccs = np.mean(librosa.feature.mfcc(y=row['data'], sr=row['sample_rate'], n_mfcc=40).T,axis=0) 
    return mfccs

In [ ]:
df['mfcc'] = df[['data','sample_rate']].apply(getMFCC,axis=1)

In [6]:
def getFFT(data,n=500):
    feature = fft(data,n)
    return feature

In [ ]:
df['fft'] = df['data'].apply(lambda x: getFFT(x))

In [7]:
def getFeatureVector(row):
    vector = np.concatenate([row['mfcc'],row['fft']]) 
    return vector

In [ ]:
df['feature_vector'] = df[['mfcc','fft']].apply(getFeatureVector,axis=1)

In [ ]:
df.head()

In [ ]:
df.to_pickle('sound_db.pkl')

In [11]:
quijote = pd.DataFrame(columns=['data','sample_rate'])
data, sample_rate = getAudio('rocio.mp3')
quijote = quijote.append({'data':data,'sample_rate':sample_rate}, ignore_index=True)
quijote['mfcc'] = quijote[['data','sample_rate']].apply(getMFCC,axis=1)
quijote['fft'] = quijote['data'].apply(lambda x: getFFT(x))
quijote['feature_vector'] = quijote[['mfcc','fft']].apply(getFeatureVector,axis=1)
quijote.to_pickle('rocio.pkl')

In [9]:
import IPython.display

In [11]:
IPython.display.Audio(data=data, rate=sample_rate)